In [87]:
import pandas as pd
import numpy as np
import torch.nn as nn 
from sklearn.datasets import make_regression
from torch.utils.data import Dataset, DataLoader
import torch
from typing import Tuple
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


In [88]:
# Make linear regression problem
X_raw, y_raw = make_regression(
    n_features=10,
    n_informative=5,
    n_samples=10000,

)
# Standard scale X and y
X_scaler = StandardScaler()
X_np = X_scaler.fit_transform(X_raw)

y_scaler = StandardScaler()
y_np = y_scaler.fit_transform(y_raw.reshape(-1,1)).flatten()

# Fit a linear regression to check coef
linear_regression = LinearRegression()
linear_regression.fit(X_np, y_np)
linear_regression.coef_

array([ 4.74936961e-01,  2.61417087e-01,  3.62929076e-02,  5.45608538e-01,
        1.58928776e-16,  6.48266616e-01,  5.89720193e-16, -8.84876241e-17,
       -1.00920016e-16,  2.44493610e-16])

In [89]:
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32)

In [90]:
X

tensor([[-0.3932, -0.2430,  1.1028,  ..., -0.6886,  0.0074, -0.7459],
        [-0.2960,  0.1223,  0.2751,  ...,  0.1748, -0.0868, -0.1728],
        [-0.5495,  2.1534,  0.4155,  ..., -0.0296, -1.3600, -1.1377],
        ...,
        [-1.6305,  0.7473,  1.4869,  ..., -0.0668, -3.9125,  0.5599],
        [ 0.9321, -0.0219,  1.3071,  ...,  0.9743, -0.1945,  0.2274],
        [ 1.3765, -0.6308, -0.6520,  ...,  0.8862,  0.4011, -1.0510]])

In [91]:
y

tensor([ 0.5427,  0.5102,  1.1947,  ..., -0.5010,  1.6198, -0.8509])

In [92]:
class Model(nn.Module):
    def __init__(
            self,
            n_features: int,
    ) -> None:
        super().__init__()
        self.fc1 = nn.Linear(
            in_features=n_features,
            out_features=1,
        )

    def forward(
        self,  
        X: torch.Tensor,  
    ) -> torch.Tensor:
        y = self.fc1(X)
        return y.flatten()
        

In [93]:
class CustomLoss(nn.Module):
    def __init__(
            self,
            threshold: float = 0.3,
            weight_max_error: float = 2,
            weight_percentage_above_threshold:float = 3,
            weight_wrong_sign: float = 1,
            sigmoid_steepness: float = 100,
    ) -> None:
        super().__init__()
        self.steepness = sigmoid_steepness
        self.threshold = threshold
        # Normalize weights and assign them
        sum_weights = (
            weight_max_error
            + weight_percentage_above_threshold
            + weight_wrong_sign
        )
        self.weight_max_error = (
            weight_max_error / sum_weights
        )
        self.weight_percentage_above_threshold = (
            weight_percentage_above_threshold / sum_weights
        )
        self.weight_wrong_sign = (
            weight_wrong_sign / sum_weights
        )

    def forward(
            self, 
            inputs: torch.Tensor, 
            targets: torch.Tensor,
        ) -> torch.Tensor:

        residuals = targets - inputs
        # Maximum abs error
        max_error = residuals.abs().max()

        # Percentage of time above threshold value
        percentage_of_time_above_x = (
            1/(1+torch.e**(-self.steepness*(residuals.abs()-self.threshold)))
        ).mean()

        # Percentage of time wrong sign
        loss_percentage_of_time_wrong_sign = (
            1/(1+torch.e**(-self.steepness*(inputs*targets)))
        )
        
        # Total loss
        total_loss = (
            self.weight_max_error * max_error
            + self.weight_percentage_above_threshold * percentage_of_time_above_x
            + self.weight_wrong_sign * loss_percentage_of_time_wrong_sign
        )
        return percentage_of_time_above_x

In [94]:
class CustomDataset(Dataset):
    def __init__(
            self,
            X: torch.Tensor,
            y: torch.Tensor,
        ) -> None:
        self.X = X
        self.y = y

    def __len__(
            self
    ) -> int:
        return self.X.shape[0]
    
    def __getitem__(
            self,
            idx: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        X_item = self.X[idx,:]
        y_item = self.y[idx]
        return X_item, y_item

In [95]:
epochs = 100
lr = 1e-4
batch_size = 10

dataloader = DataLoader(
    CustomDataset(X,y),
    batch_size=batch_size,
    shuffle=True,
)
model = Model(10)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = lr,
)
criterion = CustomLoss(
    weight_wrong_sign=1,
    weight_max_error=1,
    weight_percentage_above_threshold=1,
)
for epoch in range(epochs):
    epoch_loss = 0
    for i,(X_batch, y_batch) in enumerate(dataloader):
        prediction = model(X_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        average_loss = epoch_loss/(i+1e-10)
    print(f"Average epoch loss: {average_loss}")
    print(f"Epoch {epoch} done.")

Average epoch loss: 0.8182144329711736
Epoch 0 done.
Average epoch loss: 0.8149967318897748
Epoch 1 done.
Average epoch loss: 0.8123250277012937
Epoch 2 done.
Average epoch loss: 0.8097737606640121
Epoch 3 done.
Average epoch loss: 0.8071333277332082
Epoch 4 done.
Average epoch loss: 0.804064040845021
Epoch 5 done.
Average epoch loss: 0.8006285315936187
Epoch 6 done.
Average epoch loss: 0.7971771978161751
Epoch 7 done.
Average epoch loss: 0.7937700289027628
Epoch 8 done.
Average epoch loss: 0.7901930313657278
Epoch 9 done.
Average epoch loss: 0.7868393306676499
Epoch 10 done.
Average epoch loss: 0.7836141368528198
Epoch 11 done.
Average epoch loss: 0.7804517789824166
Epoch 12 done.
Average epoch loss: 0.777001795288959
Epoch 13 done.
Average epoch loss: 0.7728257963546823
Epoch 14 done.
Average epoch loss: 0.7684251738262076
Epoch 15 done.
Average epoch loss: 0.7638290164706178
Epoch 16 done.
Average epoch loss: 0.758393551255533
Epoch 17 done.
Average epoch loss: 0.7527521804526246
Ep

In [96]:
model.fc1.weight

Parameter containing:
tensor([[ 4.7258e-01,  2.5998e-01,  3.5086e-02,  5.4427e-01, -2.2798e-05,
          6.4136e-01,  6.4996e-04, -7.9514e-05, -4.6952e-05,  3.3786e-04]],
       requires_grad=True)

In [97]:
linear_regression.coef_

array([ 4.74936961e-01,  2.61417087e-01,  3.62929076e-02,  5.45608538e-01,
        1.58928776e-16,  6.48266616e-01,  5.89720193e-16, -8.84876241e-17,
       -1.00920016e-16,  2.44493610e-16])

# Tryout some loss functions

In [98]:
pred = torch.rand(5, requires_grad=True) - 0.5
real = torch.rand(5, requires_grad=True) - 0.5
residuals = real - pred

In [99]:
residuals

tensor([ 0.3669,  0.8406, -0.3462,  0.2298, -0.3947], grad_fn=<SubBackward0>)

In [100]:
real

tensor([ 0.3021,  0.3909, -0.3320,  0.2157,  0.0452], grad_fn=<SubBackward0>)

In [101]:
pred

tensor([-0.0648, -0.4497,  0.0142, -0.0141,  0.4399], grad_fn=<SubBackward0>)

In [102]:
steepness = 100 
threshold = 0.3

loss1 = 1/(1+torch.e**(-steepness*(residuals-threshold)))
loss1

tensor([9.9876e-01, 1.0000e+00, 8.6509e-29, 8.9671e-04, 6.7491e-31],
       grad_fn=<MulBackward0>)

In [103]:
signs = pred*real / (pred*real).abs()
loss2 = 1/(1+torch.e**(-steepness*(signs)))
loss2

tensor([0., 0., 0., 0., 1.], grad_fn=<MulBackward0>)

In [104]:
pred*real 

tensor([-0.0196, -0.1758, -0.0047, -0.0030,  0.0199], grad_fn=<MulBackward0>)